# Removing The Duplicate Poster Images In Each Movie

In [1]:
!pip install imagehash

     |████████████████████████████████| 812 kB 5.3 MB/s 
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=0d973d23b0a740850696f06711cbd38b98dd77f2ba32adbff59b50f4759a44b2
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
Successfully built imagehash


In [2]:
from PIL import Image
from tqdm.notebook import tqdm
from pathlib import Path
import imagehash
import pickle
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

movies_path = Path("movie_posters.pkl")


THRESHOLD = 20 # we chose 20 based on a random sample of duplicate posters

In [ ]:
posters = pd.read_pickle(movies_path)
posters = posters.sort_values("movie")

In [ ]:
!unzip posters.zip # the sample's posters images 

In [ ]:
# calcualte each poster image hash value

def get_img_dhash(row):
    poster_path = f'posters/{row["tconst"]}/{row["file_path"]}'
    img = Image.open(poster_path)
    row['dhash'] = imagehash.dhash(img)

    return row

posters = posters.apply(get_img_dhash, axis=1)

In [ ]:
def get_duplicates(s1, s2):
    dup = set()
    not_dup = set()
    for i, j in zip(s1,s2):
        if i not in dup and j not in dup:
            if i in not_dup:
                dup.add(j)
            elif j in not_dup:
                dup.add(i)
            else:
                dup.add(j)
                not_dup.add(i)
        else:
            if i in not_dup:
                dup.add(j)
            if j in not_dup:
                dup.add(i)
    return list(dup)


In [ ]:
posters["dups"]= False

for m, posters in tqdm(posters.groupby("tconst")):    
    dhash = [p.hash.flatten() for p in posters["dhash"]]
    dist = cdist(dhash, dhash, lambda u, v: np.count_nonzero(v != u))
    sim = np.where(dist<THRESHOLD) 
    s1 = sim[0][sim[0]!=sim[1]]
    s2 = sim[1][sim[0]!=sim[1]]
    
    dups = get_duplicates(s1, s2)
    posters.loc[posters.iloc[dups].index,'dups'] = True

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
posters = posters.reset_index(drop=True)

In [ ]:
posters.to_pickle("posters_with_dup.pkl")